http://localhost:8889/notebooks/git/google-quest-challenge/working/QstBertBase0113_1.ipynb

no freezing of bert model

# start

In [ ]:
PRFX='0326_3'

In [1]:
trntmstmp=1584412344
valtmstmp=1585069785

import datetime
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)])

grand_total=1.5e8
MIN_TM_TRN=1580947200
MIN_TM_TST=1581552000
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (MIN_TM_TRN, MIN_TM_TST)])


SEED=101

['2020-03-17 02:32:24', '2020-03-24 17:09:45']
['2020-02-06 00:00:00', '2020-02-13 00:00:00']


In [2]:
!nvidia-smi

Fri Mar 27 01:55:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   78C    P0   148W / 149W |   8168MiB / 11441MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# setup

In [24]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'

import torch
from transformers import *
import torch
device=torch.device('cpu')
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


pretrained_weights='bert-base-multilingual-cased'
bertmodel = BertModel.from_pretrained(pretrained_weights, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)

import random
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(SEED)

In [4]:
cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwng_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_fllwer_cnt',
'u2_fllwng_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]
cols_cat = ['twttyp','lang']
cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)


tgt2col=dict(zip(tgts,cols_tgt))
tgt2col

{'Retweet': 'retwt',
 'Reply': 'reply',
 'Like': 'like',
 'RTwCmnt': 'retwt_cmmnt'}

# prep

In [29]:
%%time
dftrn=pd.read_csv(
    f'{p_in}/trn_{trntmstmp}.tsv',
    sep='\x01', header=None, 
    encoding='utf-8', 
    names=cols, 
    nrows=10
)

CPU times: user 8.83 ms, sys: 812 µs, total: 9.64 ms
Wall time: 8.2 ms


In [30]:
# lens = dftrn.toks.apply(lambda x: len(x.split('\t')))

# lens.mean(), np.percentile(lens, 50), np.percentile(lens, 95), np.percentile(lens, 99)
# (46.754583, 41.0, 106.0, 135.0)


In [31]:
# tokenizer.pad_token, tokenizer.pad_token_id
# ('[PAD]', 0)

# tokenizer.sep_token, tokenizer.sep_token_id
# ('[SEP]', 102)
maxlen=128
def mkids(x):
    tokids=list(map(int, x.split('\t')))
    l=len(tokids) 
    if l<=maxlen: 
        return tokids + [0]*(maxlen-len(tokids))
    else: 
        return tokids[:maxlen-1]+[102]

In [32]:
def mk_tensors(df, istrn=True):
    tokids=dftrn.toks.apply(lambda x: mkids(x))
    Xarr=np.array(list(tokids))
    X=torch.tensor(Xarr,dtype=torch.long)
    if not istrn: return X
    ys=dftrn[cols_tgt_tmstmp].notna().values
    ys=torch.tensor(ys,dtype=torch.float)
    return X,ys
    

In [33]:
X,ys = mk_tensors(dftrn)

In [34]:
BS=2
ds = TensorDataset(X,ys)
dl = DataLoader(ds, batch_size=BS, shuffle=True)
for step, batch in enumerate(dl):
    X_b,ys_b = (o.to(device) for o in batch)
    print(X_b.shape,ys_b.shape)
    break

torch.Size([2, 128]) torch.Size([2, 4])


# model

In [35]:
EPOCHS=2#4
WD=0.01
LR=1e-5
SCHDLR_FUNC = get_cosine_schedule_with_warmup 
WARMUP_RATE = 0.05
N_CYCLS = .5
EPS = 1e-6


In [36]:
# ds = TensorDataset(X,ys)
# dl = DataLoader(ds, batch_size=BS, shuffle=True)
# for step, batch in enumerate(dl):
#     X_b,ys_b = (o.to(device) for o in batch)
#     print(X_b.shape,ys_b.shape)
#     break

# # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
# bertmodel=bertmodel.eval()
# bertmodel.to(device)
# with torch.no_grad():
#     last_hidden_state, pooler_output, hidden_states = bertmodel(X_b,)
#     avg_pool = torch.mean(last_hidden_state,1)
#     max_pool,_ = torch.max(last_hidden_state,1)
# last_hidden_state.shape,pooler_output.shape, len(hidden_states), avg_pool.shape, max_pool.shape
# (torch.Size([2, 128, 768]),
#  torch.Size([2, 768]),
#  13,
#  torch.Size([2, 768]),
#  torch.Size([2, 768]))

In [37]:
# see RobertaClassificationHead in transformers/modeling_roberta.py
N_HIDDEN = 768 
class TwtModel(nn.Module):
    def __init__(self, bertmodel, num_labels=4):
        super(TwtModel, self).__init__()
        self.bertmodel = bertmodel        
        dense_size = 64
        self.relu=nn.ReLU(inplace=True)
        nx = N_HIDDEN*2
        self.dense = nn.Linear(nx, nx)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(nx, num_labels)
        
    def forward(self, x):
        attn_msk = (x!=tokenizer.pad_token_id).float().to(device)
        segments = torch.zeros(x.shape, dtype=torch.long).to(device)
        bert_output=self.bertmodel(x,attention_mask=attn_msk,token_type_ids=segments)
        last_hidden_state,pooler_output,hidden_states = bert_output
        avg_pool = torch.mean(last_hidden_state,1)
        max_pool,_ = torch.max(last_hidden_state,1)
        x = torch.cat([avg_pool,max_pool],1) 
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

# train

In [38]:
msk_vl = np.random.rand(len(X))<0.15
tr = np.where(~msk_vl)[0]
vl = np.where( msk_vl)[0]
Xtr,Xvl = X[tr],X[vl]
ys_tr, ys_vl = ys[tr], ys[vl]
ds_tr = TensorDataset(Xtr,ys_tr)
ds_vl = TensorDataset(Xvl,ys_vl)
dl_tr = DataLoader(ds_tr, batch_size=BS,   pin_memory=True, shuffle=True)
dl_vl = DataLoader(ds_vl, batch_size=BS, pin_memory=True, shuffle=False)

In [39]:
model = TwtModel(bertmodel)
model = model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': WD},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=EPS)
t_total = int(EPOCHS*len(dl))
scheduler = SCHDLR_FUNC(
    optimizer, 
    num_warmup_steps=int(WARMUP_RATE*t_total), 
    num_training_steps=t_total,
    num_cycles=N_CYCLS,
)

In [40]:
def evaluate(model, dl):
    y_pred = []
    y_true = []
    losses = []
    model.eval()
    for step, dat_b in enumerate(dl):
        model.eval()
        dat_b = (o.to(device) for o in dat_b)
        (*xs_b,y_b)=dat_b
        if len(y_b)<2: continue #ignore last batch if size is 1
        yb_pred = model(xs_b)
        y_true.append(y_b.cpu().detach().numpy())
        y_pred.append(yb_pred.cpu().detach().numpy())
        loss_ = F.binary_cross_entropy_with_logits(yb_pred,y_b).item()
        losses.append(loss_)
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    scrs = spearmanrs(y_true, y_pred)
    loss = np.mean(losses)
    return y_pred, y_true, loss, scrs

In [41]:
model.zero_grad()
set_seed(SEED)

epc2loss_tr = []
epc2loss_vl = []
epc2scrs_tr = []
epc2scrs_vl = []
best_scr = float('-inf')
save_p = f'{p_out}/fld_{fld}.p'
for epoch in range(EPOCHS):
    print(dtnow(), f'epoch {epoch} starts')
    y_tr_epoch = []
    y_pred_tr_epoch = []
    for step, dat_b in enumerate(dl):
        model.train()
        x_b,ys_b = (o.to(device) for o in batch)
        yb_pred = model(x_b)
        y_tr_epoch.append(y_bs.cpu().detach().numpy())
        y_pred_tr_epoch.append(yb_pred.cpu().detach().numpy())
        loss =  F.binary_cross_entropy_with_logits(yb_pred,ys_b)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()

    y_pred_tr_epoch = np.concatenate(y_pred_tr_epoch)
    y_tr_epoch = np.concatenate(y_tr_epoch)
    
    break
    
#     scrs_tr = spearmanrs(y_tr_epoch, y_pred_tr_epoch)
#     scr_tr = np.mean(scrs_tr)
#     pred_vl, y_true_vl, lss_vl, scrs_vl = evaluate(model, X_vl,Xcat_vl,Xhst_vl,Xfeat_vl,y_vl)
#     scr_vl = np.mean(scrs_vl)
#     print(f'lss_tr: {lss_tr: .4f}; scr_tr: {scr_tr: .4f}; ')
#     print(f'lss_vl: {lss_vl: .4f}; scr_vl: {scr_vl: .4f}; ')
#     if scr_vl>best_scr: 
#         best_scr = scr_vl
#         best_pred_vl = pred_vl
#         print(f'found better scr, saving model...')
#         torch.save(model.state_dict(),save_p)

#     epc2loss_tr.append(lss_tr)
#     epc2loss_vl.append(lss_vl)
#     epc2scrs_tr.append(scrs_tr)
#     epc2scrs_vl.append(scrs_vl)

# fld2loss_tr.append(epc2loss_tr)
# fld2loss_vl.append(epc2loss_vl)
# fld2scrs_tr.append(epc2scrs_tr)
# fld2scrs_vl.append(epc2scrs_vl)
# preds_vl += list(best_pred_vl)
# ys_true_vl += list(y_true_vl)

# del model
# gc.collect()

# pickle.dump(vls, open(f'{p_out}/vls.p', 'wb'))
# pickle.dump(preds_vl, open(f'{p_out}/preds_vl.p', 'wb'))
# pickle.dump(ys_true_vl, open(f'{p_out}/ys_true_vl.p', 'wb'))

# results = {    
# 'fld2loss_tr': np.array(fld2loss_tr),
# 'fld2loss_vl': np.array(fld2loss_vl),
# 'fld2scrs_tr': np.array(fld2scrs_tr),
# 'fld2scrs_vl': np.array(fld2scrs_vl),
# }
# return results

NameError: name 'p_out' is not defined